In [1]:
import pandas as pd
import glob
import utils
import json
from functools import reduce

DATA = "./geojson/"

In [2]:
files = glob.glob(f"{DATA}municipios*")

In [3]:
files

['./geojson\\municipios_canarias.geojson',
 './geojson\\municipios_peninsula.geojson']

In [4]:
with open(files[0], "r") as test:
    test2 = json.load(test)

In [5]:
with open(f"{DATA}/test.geojson", "w", encoding="utf-8") as fp:
    json.dump(test2, fp)

In [6]:
with open(f"{DATA}/test.geojson", "r") as test:
    test3 = json.load(test)
test2 == test3


True

In [11]:
with open(files[0], "r") as file:
    merged_json = json.load(file)

municipios = []
for file in files:
    with open(file, "r", encoding="utf8") as json_file:
        municipios.append(json.load(json_file)['features'])
        
merged_json['features'] = reduce(lambda a, b: a+b, municipios)

In [12]:
with open(f"{DATA}total_municipios.geojson", "w") as fp:
    json.dump(merged_json, fp)

In [13]:
df = pd.DataFrame.from_records([elem['properties'] for elem in merged_json['features']])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8219 entries, 0 to 8218
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   comunidad_name            8219 non-null   object 
 1   comunidad_normalizedName  8219 non-null   object 
 2   normalizedName            8219 non-null   object 
 3   name                      8219 non-null   object 
 4   provincia_name            8219 non-null   object 
 5   comunidad_id              94 non-null     float64
 6   provincia_normalizedName  8219 non-null   object 
 7   provincia_id              8219 non-null   int64  
 8   id                        8219 non-null   int64  
 9   comunidad_id_indra        8219 non-null   int64  
 10  layer                     94 non-null     object 
 11  path                      94 non-null     object 
 12  fid                       8125 non-null   float64
 13  comunida_id               8125 non-null   float64
dtypes: float

In [15]:
df.head()

,comunidad_name,comunidad_normalizedName,normalizedName,name,provincia_name,comunidad_id,provincia_normalizedName,provincia_id,id,comunidad_id_indra,layer,path,fid,comunida_id
0,Canarias,canarias,agaete,Agaete,Las Palmas,5.0,las-palmas,35,1,5,las-palmas provincias_peninsula,E:/eclipse-rtve/rtve-app-elecciones/public/mul...,NaN,NaN
1,Canarias,canarias,aguimes,Agüimes,Las Palmas,5.0,las-palmas,35,2,5,las-palmas provincias_peninsula,E:/eclipse-rtve/rtve-app-elecciones/public/mul...,NaN,NaN
2,Canarias,canarias,antigua,Antigua,Las Palmas,5.0,las-palmas,35,3,5,las-palmas provincias_peninsula,E:/eclipse-rtve/rtve-app-elecciones/public/mul...,NaN,NaN
3,Canarias,canarias,arrecife,Arrecife,Las Palmas,5.0,las-palmas,35,4,5,las-palmas provincias_peninsula,E:/eclipse-rtve/rtve-app-elecciones/public/mul...,NaN,NaN
4,Canarias,canarias,artenara,Artenara,Las Palmas,5.0,las-palmas,35,5,5,las-palmas provincias_peninsula,E:/eclipse-rtve/rtve-app-elecciones/public/mul...,NaN,NaN
